In [3]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from skimage.feature import hog
from skimage import color

# ==========================
# 1) HÀM ĐỌC ẢNH & GÁN NHÃN
# ==========================
def load_data_from_folder(base_path, img_size=(64,64)):
    X, y = [], []
    classes = os.listdir(base_path)
    for label in classes:
        class_path = os.path.join(base_path, label)
        if not os.path.isdir(class_path):
            continue
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            img = cv2.imread(file_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# ==========================
# 2) TRÍCH HOG FEATURE
# ==========================
def extract_hog_features(images):
    features = []
    for img in images:
        gray = color.rgb2gray(img)  # chuyển grayscale
        hog_feat = hog(gray, 
                       pixels_per_cell=(8,8), 
                       cells_per_block=(2,2), 
                       feature_vector=True)
        features.append(hog_feat)
    return np.array(features)

# ==========================
# 3) LOAD DỮ LIỆU
# ==========================
train_dir = r"D:\BT_KNN\Rock-Paper-Scissors\train"
test_dir  = r"D:\BT_KNN\Rock-Paper-Scissors\test"

X_train, y_train = load_data_from_folder(train_dir)
X_test,  y_test  = load_data_from_folder(test_dir)

print("Raw Train shape:", X_train.shape)
print("Raw Test shape:", X_test.shape)

# encode nhãn
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

# tách train thành train + val
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# ==========================
# 4) TRÍCH HOG
# ==========================
X_train_hog = extract_hog_features(X_train)
X_val_hog   = extract_hog_features(X_val)
X_test_hog  = extract_hog_features(X_test)

print("HOG Train shape:", X_train_hog.shape)

# ==========================
# 5) CHUẨN HÓA + PCA
# ==========================
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_val_hog   = scaler.transform(X_val_hog)
X_test_hog  = scaler.transform(X_test_hog)

pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_hog)
X_val_pca   = pca.transform(X_val_hog)
X_test_pca  = pca.transform(X_test_hog)

print("PCA Train shape:", X_train_pca.shape)

# ==========================
# 6) GRID SEARCH KNN
# ==========================
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid.fit(X_train_pca, y_train)

print("Best Params:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

# ==========================
# 7) ĐÁNH GIÁ
# ==========================
best_knn = grid.best_estimator_

y_val_pred = best_knn.predict(X_val_pca)
print("\nValidation Accuracy:", accuracy_score(y_val, y_val_pred))

y_test_pred = best_knn.predict(X_test_pca)
print("\nTest Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred, target_names=le.classes_))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Raw Train shape: (2520, 64, 64, 3)
Raw Test shape: (372, 64, 64, 3)
HOG Train shape: (2016, 1764)
PCA Train shape: (2016, 100)
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best CV Score: 1.0

Validation Accuracy: 1.0

Test Accuracy: 0.7043010752688172

Classification Report:
               precision    recall  f1-score   support

       paper       0.62      0.66      0.64       124
        rock       0.84      0.86      0.85       124
    scissors       0.65      0.59      0.62       124

    accuracy                           0.70       372
   macro avg       0.70      0.70      0.70       372
weighted avg       0.70      0.70      0.70       372


Confusion Matrix:
 [[ 82   4  38]
 [ 16 107   1]
 [ 35  16  73]]


In [4]:
def predict_single_image(img_path, img_size=(64,64)):
    # đọc ảnh
    img = cv2.imread(img_path)
    if img is None:
        print("❌ Không đọc được ảnh:", img_path)
        return
    img = cv2.resize(img, img_size)

    # trích HOG
    gray = color.rgb2gray(img)
    hog_feat = hog(gray, pixels_per_cell=(8,8), cells_per_block=(2,2), feature_vector=True)

    # chuẩn hóa + PCA
    hog_feat = scaler.transform([hog_feat])
    hog_feat = pca.transform(hog_feat)

    # dự đoán
    pred = best_knn.predict(hog_feat)[0]
    label = le.inverse_transform([pred])[0]

    # hiển thị
    cv2.imshow(f"Prediction: {label}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return label


In [24]:
# ví dụ test một ảnh bất kỳ
img_path = r"D:\BT_KNN\Rock-Paper-Scissors\test\rock\testrock04-27.png"
label = predict_single_image(img_path)
print("Ảnh dự đoán là:", label)


Ảnh dự đoán là: rock


In [18]:
# ví dụ test một ảnh bất kỳ
img_path = r"Bua.jpg"
label = predict_single_image(img_path)
print("Ảnh dự đoán là:", label)

Ảnh dự đoán là: rock


In [26]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from skimage.feature import hog
from skimage import color

# ==========================
# 1) HÀM ĐỌC ẢNH & GÁN NHÃN
# ==========================
def load_data_from_folder(base_path, img_size=(64,64)):
    X, y = [], []
    classes = os.listdir(base_path)
    for label in classes:
        class_path = os.path.join(base_path, label)
        if not os.path.isdir(class_path):
            continue
        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            img = cv2.imread(file_path)
            if img is None:
                continue
            img = cv2.resize(img, img_size)
            X.append(img)
            y.append(label)
    return np.array(X), np.array(y)

# ==========================
# 2) TRÍCH HOG FEATURE
# ==========================
def extract_hog_features(images):
    features = []
    for img in images:
        gray = color.rgb2gray(img)  # chuyển grayscale
        hog_feat = hog(gray,
                       pixels_per_cell=(8,8),
                       cells_per_block=(2,2),
                       feature_vector=True)
        features.append(hog_feat)
    return np.array(features)

# ==========================
# 3) LOAD DỮ LIỆU ARCHIVE
# ==========================
archive_dir = r"archive"  # chứa paper/rock/scissors

X, y = load_data_from_folder(archive_dir)
print("Raw data shape:", X.shape)

# encode nhãn
le = LabelEncoder()
y = le.fit_transform(y)

# chia train (70%), val (15%), test (15%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# ==========================
# 4) TRÍCH HOG
# ==========================
X_train_hog = extract_hog_features(X_train)
X_val_hog   = extract_hog_features(X_val)
X_test_hog  = extract_hog_features(X_test)

print("HOG Train shape:", X_train_hog.shape)

# ==========================
# 5) CHUẨN HÓA + PCA
# ==========================
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_val_hog   = scaler.transform(X_val_hog)
X_test_hog  = scaler.transform(X_test_hog)

pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_hog)
X_val_pca   = pca.transform(X_val_hog)
X_test_pca  = pca.transform(X_test_hog)

print("PCA Train shape:", X_train_pca.shape)

# ==========================
# 6) GRID SEARCH KNN (dùng train + val)
# ==========================
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv=3, scoring='accuracy', verbose=1, n_jobs=-1)
grid.fit(X_train_pca, y_train)

print("Best Params:", grid.best_params_)
print("Best CV Score:", grid.best_score_)

# ==========================
# 7) ĐÁNH GIÁ
# ==========================
best_knn = grid.best_estimator_

# Validation
y_val_pred = best_knn.predict(X_val_pca)
print("\nValidation Accuracy:", accuracy_score(y_val, y_val_pred))

# Test
y_test_pred = best_knn.predict(X_test_pca)
print("\nTest Accuracy:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred, target_names=le.classes_))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))


Raw data shape: (2188, 64, 64, 3)
Train: (1531, 64, 64, 3) Val: (328, 64, 64, 3) Test: (329, 64, 64, 3)
HOG Train shape: (1531, 1764)
PCA Train shape: (1531, 100)
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best CV Score: 0.9803998311653429

Validation Accuracy: 0.9908536585365854

Test Accuracy: 0.9787234042553191

Classification Report:
               precision    recall  f1-score   support

       paper       0.95      0.98      0.97       107
        rock       0.99      1.00      1.00       109
    scissors       0.99      0.96      0.97       113

    accuracy                           0.98       329
   macro avg       0.98      0.98      0.98       329
weighted avg       0.98      0.98      0.98       329


Confusion Matrix:
 [[105   1   1]
 [  0 109   0]
 [  5   0 108]]


In [27]:
def predict_single_image(img_path, img_size=(64,64)):
    # đọc ảnh
    img = cv2.imread(img_path)
    if img is None:
        print("❌ Không đọc được ảnh:", img_path)
        return None
    img = cv2.resize(img, img_size)

    # trích HOG
    gray = color.rgb2gray(img)
    hog_feat = hog(gray, pixels_per_cell=(8,8), cells_per_block=(2,2), feature_vector=True)

    # chuẩn hóa + PCA (dùng scaler & pca đã fit từ train)
    hog_feat = scaler.transform([hog_feat])
    hog_feat = pca.transform(hog_feat)

    # dự đoán
    pred = best_knn.predict(hog_feat)[0]
    label = le.inverse_transform([pred])[0]

    # hiển thị kết quả
    print(f"Ảnh {os.path.basename(img_path)} dự đoán là 👉 {label}")
    return label


In [46]:
# test ảnh ngoài dataset
img_path = r"Q2.jpg"   # đổi thành đường dẫn thực tế của bạn
predict_single_image(img_path)

Ảnh Q2.jpg dự đoán là 👉 rock


np.str_('rock')